# Class 3: Pattern matching in sequence

---
# Before Class
Class today is a basic intro into reading files and processing them in Python that will be necessary for the rest of the class.
Prior to class, please do the following:
1. Install `wget` in your b529 environment using: `$ conda install wget`
1. Review the FASTA and GFF formats
1. Review or read up on Python syntax for:
  1. Reading files using open() and gzip.open()
  1. Generators
  1. Substrings using slicing


---
## Learning Objectives

1. Downloading files programmatically
* Read & process FASTA files
* Read & process GTF files
* Pattern matching

---
## Background

Bacteria use pattern matching in a variety of processes. These include the use of restriction enzymes that cut DNA at exact sequences as well as other factors that allow for more flexibility in the pattern match. A current popular example of this is CRISPR-Cas9 where a 20bp sequence is used to scan the genome and cut DNA. One of the earliest examples of known patterns in bacteria is the Shine-Dalgarno sequence. This pattern was identified in the mid-70s in _E. coli_ where the 3' end of the 16S rRNA sequence was found to recongize a complementary sequence (AGGAGGU) upstream of the start codon (AUG). This sequence has been shown to be important for the initiation of translation by bacterial ribosomes.

Our goal today will be to identify genes with the exact matching sequence AGGAGGU in the 50bp upstream of genes in the _Bacillus subtilis_ genome.

### FASTA Format


Recall from the lecture slides that the FASTA format (typically with the file extension .fa) has the following format:


### GFF Format

Also recall from the slide that the GFF format (or Generic Feature Format) is used to annotate features in the genome. For our use case here, we will be using an annotation of the genome of _B. subtilis_ from NCBI to identify the locations of coding sequence. GFF has the following format:

---
## Imports

In [1]:
import sys
import os
import gzip

---
## Get the data


Go to NCBI and search for Bacillus subtilis subtilis 168 using the 'Genome' dropdown and copy the links to both the genome and GFF annotation file.

Now use `wget` to download the files into the /data/ folder.

You will need to install `wget` on your sysytem using `conda install wget`.

<center><img src='./figures/bsubtilis_screenshot.png'/></center>

---
## Implement FASTA and GFF reader

We first need to build generic functions for handling sequence data (FASTA) and annotations of coordinates on that sequence (GFF). Because the _B. subtilis_ genome fasta file only has a single entry, we have provided an example fasta file to make sure that your code handles multiple sequences appropriately (data/example.fa).

In [2]:
#FASTA READER

def get_fasta(file):
    """Generator to lazily get all the fasta entries from a fasta file

    Args:
        fasta_file (str): /path/and/name/to.fa[.gz] 
            (file may be gziped)

    Yields:
        header (str): header sequence of fasta entry (excludes '>')
        seq (str): concatenated string sequence of the fasta entry
    """
    
    if '.gz' in file:
        file_object = gzip.open(file, 'rt')
    else:
        file_object = open(file, 'rt')
        
    name=''
    seq=''
    for line in file_object:
        
        # Capture the next header, report what we have, and update
        if line.startswith('>') and seq: #not first seq
            name = name[1:] #removes the carrot
            yield name, seq
            name=line.strip()
            seq=''
            
        # Get to the first header
        elif line.startswith('>'):  #first seq
            name=line.strip()
            
        # Just add sequence if it is the only thing there
        else:
            seq+=line.strip()
            
    # At the end, return the last entries
    if name and seq: #last seq
            name = name[1:]
            yield name, seq


In [3]:
#This will test your FASTA READER function

#file_name="data/GCF_000009045.1_ASM904v1_genomic.fna.gz"
file_name="data/example.fa.gz"
for name, seq in get_fasta(file_name):
    print (name, seq)

seq0 GGCAGATTCCCCCTAGACCCGCCCGCACCATGGTCAGGCATGCCCCTCCTCATCGCTGGGCACAGCCCAGAGGGTATAAACAGTGC
seq1 ATGAGAGCCCTCACACTCCTCGCCCTATTGGCCCTGGCCGCACTTTGCATCGCTGGCCAGGCAGGTGAGTGCCCCCACCTCCCCTCAGGCCGCATTGCAGTGGGGGCTGAGAGGAGGA
seq2 GAGAGGAGGGAAGAGCAAGCTGCCCGAGACGCAGGGGAAGGAGGATGAGGGCCCTGGGGATGAGCTGGGGTGAACCAGGCTCCCTTTCCTTTGCAGGTGCGA


In [4]:
#GFF READER
from functools import total_ordering

@total_ordering
class GffEntry:
    """Main class for handling GFF entries
    
    Truncates the GFF entry to required data only. Also, class is totally ordered.
    This means that comparison operators can be used on GffEntry objects
    
    Attributes:
        seqid (str): The contig that the GFF entry is associated with
        type (str): The type of object that the GFF entry classifies as
        start (int): The left-most nucleotide position of the GFF entry relative to seqid (1-indexed)
        end (int): The right-most nucleotide position of the GFF entry relative to seqid (1-indexed)
        strand (str): Whether the entry is on the forward (+), backward (-) strand or N/A (.)
    """
    
    slots = 'seqid type start end strand'.split()
    
    def __init__(self, args):
        """Initialize the object.
        
        Aggregates all GFF entry columns, and selectively assigns them to attributes
        
        Args:
            args (list): the complete stripped and split GFF entry line
        """
        self.seqid = args[0]
        self.type = args[2]
        self.start = int(args[3])
        self.end = int(args[4])
        self.strand = args[6]
    
    def __str__(self):
        """Determines how GffEntry appear when `print()` is called on them"""
        return f'{self.seqid}\t{self.type}\t{self.start}\t{self.end}\t{self.strand}'
    
    def __len__(self):
        """Determines how GffEntry reports when `len()` is called on them"""
        return self.end - self.start

    def __eq__(self, other):
        self_check = (self.seqid, self.type, self.start, self.end, self.strand)
        other_check = (other.seqid, other.type, other.start, other.end, other.strand)
        if  self_check == other_check:
            return True
    
    def __lt__(self, other):
        if self.seqid < other.seqid:
            return True
        elif self.seqid == other.seqid:
            if self.start < other.start:
                return True
            elif self.start == other.start:
                if self.end < other.end:
                    return True
        

def get_gff(gff_file):
    """Generator that lazily reports each of the GFF entries within the GFF file
    
    Args:
        gff_file (str): /path/and/name/to.gff[.gz]
    
    Yields:
        (GffEntry): A GFF entry object with the attributes of seqid, type, start, end, and strand
    """
    if '.gz' in gff_file:
        gff_file = gzip.open(gff_file, 'rb')
    else:
        gff_file = open(gff_file, 'rb')
    for entry in gff_file:
        entry = entry.decode('ascii')
        if entry.startswith('#') or not entry:
            continue
        yield GffEntry(entry.strip().split('\t'))
        

In [ ]:
#This will test your GFF READER function
file_name="data/GCF_000009045.1_ASM904v1_genomic.gff.gz"
for gff_entry in get_gff(file_name):
    print(gff_entry)


---
## Extract promoter regions

Given a geneome and a list of genes, we are interested in studying the promoter regions. We now need to write a function for extracting sequence from genomic coordinates. Specifically, we would like to extract the 50bp upstream of the TSS for every gene. Because we are looking for the region upstream of the TSS, use the 'CDS' annotation from the GFF file. Remember that for genes on the - strand you will need to take the reverse complement! 

In [6]:
def reverse_complement(seq):
    """Get the reverse complement of a nucleotide sequence

    Returns the reverse complement of the input string representing a DNA 
    sequence. Works only with DNA sequences consisting solely of  A, C, G, T or N 
    characters. Preserves the case of the input sequence.

    Args:
        seq (str): a DNA sequence string

    Returns:
        (str): The reverse complement of the input DNA sequence string.
    """
    
    # Used to easily translate strings
    compStrDNA = str.maketrans('ACGTacgt', 'TGCAtcga')

    # Translate then reverse seq
    return seq.translate(compStrDNA)[::-1]


In [7]:
def get_seq(seq, start, end, strand, size):
    """Get the desired sub sequence from genomic coordinates
    
    Args:
        seq (str): nucleotide sequence
        start (int): left-most desired nucleotide sequence
        end (int): right-most desired nucleotide sequence
        strand (str): Whether the entry is on the forward (+), backward (-) strand or N/A (.)
        size (int): how far upstream to get extra sequence (default: 50)
    
    Returns:
        promoter_seq (str): the desired sub-sequence of seq at coordinates start-end corrected to + strand
    """
    promoter_seq = ''

    if strand == '-':
        promoter_seq = reverse_complement(seq[end:(end+size)])
    else :
        promoter_seq = seq[(start-size-1):(start-1)]

    return promoter_seq


In [ ]:
# Now for the sequence from our genome, we will output the promoter sequences from each region

seq_file="data/GCF_000009045.1_ASM904v1_genomic.fna.gz"
gff_file="data/GCF_000009045.1_ASM904v1_genomic.gff.gz"

for name, seq in get_fasta(seq_file):
    for gff_entry in get_gff(gff_file):
        if gff_entry.type == 'CDS':
            promoter_seq = get_seq(seq, gff_entry.start, gff_entry.end, gff_entry.strand, 50)
            print (promoter_seq)



---
## Use pattern matching to locate Shine-Dalgarno sequences

Finally, we will do very basic pattern matching using regular expressions. Using the previously implemented code, count how many of the regions have the Shine-Dalgarno sequences in the promoter (AGGAGGT). Note: You have the option of using the RE (Regular Expressions) package here to describe more complex patterns than a simple string.

In [9]:
seq_file="data/GCF_000009045.1_ASM904v1_genomic.fna.gz"
gff_file="data/GCF_000009045.1_ASM904v1_genomic.gff.gz"

hits = 0
total = 0

for name, seq in get_fasta(seq_file):
    for gff_entry in get_gff(gff_file):
        if gff_entry.type == 'CDS':
            promoter_seq = get_seq(seq, gff_entry.start, gff_entry.end, gff_entry.strand, 50)
            if "AGGAGGT" in promoter_seq:
                hits+=1
            total+=1

print (hits, total)


212 4328
